## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from openai import AsyncOpenAI
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from redmail import gmail
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown
from ddgs import DDGS

In [2]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [3]:
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv('GROQ_API_KEY')
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
groq_openai_model = OpenAIChatCompletionsModel(model="openai/gpt-oss-120b", openai_client=groq_client)

In [4]:
# 1. Define a manual search tool
@function_tool
async def web_search(query: str) -> str:
    """
    Searches the web for the given query and returns a summary of results.
    """
    with DDGS() as ddgs:
        results = [r['body'] for r in ddgs.text(query, max_results=5)]
        return "\n\n".join(results)

In [5]:
INSTRUCTIONS = "You are a research assitant. Given a search term, you use the 'web_search' tool to do a deep search on the web for the term and provide a summary"

search_agent = Agent(
    name="Search Worker",
    instructions=INSTRUCTIONS,
    tools=[web_search],
    model=groq_openai_model,
    model_settings=ModelSettings(tool_choice="required"),
)

In [6]:
master_tools = [search_agent.as_tool(tool_name="Gorq_Search_Worker", tool_description="Call this to search for ONE specific query from your list. Do not send multiple queries at once.")]

In [7]:
MASTER_INSTRUCTIONS = """
You are a post-doc level lead researcher. Your task:
1. Break the user's reseraach subject into at least 3 and maximum of 10 distinct search queries.
2. For EACH query, call 'Groq_Search_Worker' tool. Do not send multiple queries at once.
3. Once all searches are done, gather data and at the end produce a concise summary of the results. The summary must 2-3 paragraphs and less than 500 
words. Write succintly. This will be consumed by someone synthesizing a report, so it's vital you capture the 
essence and ignore any fluff. Do not include any additional commentary other than the summary itself.
"""
master_planner = Agent(name="Master Planner", instructions=MASTER_INSTRUCTIONS, model=groq_openai_model, tools=master_tools)

In [8]:
message = "Latest AI Agent frameworks in 2025 and what might become the dominant in 2026"

with trace("Search"):
    result = await Runner.run(master_planner, message)

display(Markdown(result.final_output))

The AI‑agent landscape in 2025 matured from isolated scripts to interoperable, graph‑based ecosystems that combine prompting, tool‑calling, memory, and multimodal capabilities. The most widely adopted frameworks—LangChain + LangGraph, Microsoft Semantic Kernel (SK), LlamaIndex 2.0, CrewAI, AutoGPT‑Open, OpenAI Assistants SDK, Anthropic Prompt Engine, Haystack, Mistral‑AI Agent SDK, and the Rust‑based Open‑Source Agentic‑Kit—cover a spectrum from open‑source community hubs to enterprise‑grade, cloud‑native stacks. Adoption signals (GitHub stars, PyPI downloads, enterprise contracts, and cloud‑service integrations) show LangChain + LangGraph leading with ~150 k stars and 3 M monthly downloads, followed by Semantic Kernel (deep Azure integration) and LlamaIndex (data‑centric RAG). Multi‑agent collaboration frameworks such as CrewAI and AutoGen introduced task‑allocation, negotiation protocols, and shared memory, while safety sandboxes and policy engines became standard to satisfy regulatory demands. Low‑code tools (AutoGPT‑Next) broadened the user base, and plug‑in marketplaces accelerated reuse across domains.

Forecasts for 2026 predict a consolidated market where three ADL‑compliant stacks dominate: **LangChain + LlamaIndex (“LangChain‑Index” stack)** (~32 % share), **Microsoft Semantic Kernel** (~21 %), and **AutoGen/OpenAI Assistants API** (~18 %). These frameworks benefit from emerging standards—the Agent Definition Language (ADL), OpenAI Function Calling v2, and the LLM‑Orchestration Interoperability (LOI) RFC—enabling agents defined once to run on any major cloud provider (Azure, AWS Bedrock, Google Vertex). Semantic Kernel captures regulated‑industry spend through built‑in policy‑as‑code, while LangChain‑Index excels in data‑grounded workflows, and AutoGen/Assistants offers managed, cost‑predictable multi‑agent services for SaaS co‑pilot products.

For stakeholders: enterprises should pilot Semantic Kernel for Azure‑centric, compliance‑heavy workloads and adopt LangChain‑Index for hybrid data‑centric agents; start‑ups aiming for rapid market entry should build on AutoGen + OpenAI Assistants and maintain an ADL‑compatible fallback to LangChain; investors should focus on LangChain Inc., CrewAI Labs, and Semantic Kernel‑backed startups. Aligning development with ADL and the upcoming LOI standard will ensure portability across managed cloud agent services and protect against vendor lock‑in, positioning adopters to capture the fastest‑growing segment of the AI market, projected to reach $5.2 B by 2026.

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [9]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [10]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [11]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most recent advancements and frameworks in AI for 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore various AI frameworks and their applications in 2025.', query='top AI frameworks for intelligent agents 2025'), WebSearchItem(reason='To see expert opinions and analyses on future AI frameworks and developments.', query='AI agent frameworks trends 2025')]


In [12]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("ed.donner@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"


@function_tool
def send_html_email(subject: str, message_body: str):
    """ Send out an HTML email with an embedded image to all sales prospects """

    html_template = f"""
    <div style="font-family: Jost, sans-serif; padding: 20px;">
        <h2>{subject}</h2>
        <p>{message_body}</p>
        <br>
        <hr>
        <img src="{{{{ my_logo.src }}}}" style="width: 150px;">
    </div>
    """

    gmail.username = "emasnavi1@gmail.com"
    gmail.password = os.environ.get('GMAIL_APP_PASSWORD')
    gmail.send(
        subject=subject,
        receivers=["ehsan.masnavi@gmail.com"],
        # 1. Define the HTML with the special syntax
        html=html_template,
        # 2. Tell Redmail where the file is
        body_images={
            "my_logo": "./me/emasnavi.png" 
        }
    )
    return {"status": "success"}

In [13]:
#  send_email
send_html_email

FunctionTool(name='send_html_email', description='Send out an HTML email with an embedded image to all sales prospects', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'message_body': {'title': 'Message Body', 'type': 'string'}}, 'required': ['subject', 'message_body'], 'title': 'send_html_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015D7DF9B420>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [27]:
email_writer_instructions = """ 
You are able to come up with a good engaging subject, and body of an email based on a detailed report you receive.
Include the entire report in the email. The body of the email will be inserted into a <body> segment of an html temlpate. so 
ensure the body of the email is html compatible.
"""

class EmailContent(BaseModel):
    subject: str= Field(description="A proper subject for the email to be sent based on a detail report received.")
    message_body: str  = Field(description="A proper body for the email to be sent based on a detail report received. include the entire detailed report.")
    
email_writer_agent = Agent(name="Email Writer", instructions=email_writer_instructions, model="gpt-4o-mini", output_type=EmailContent)

In [28]:
INSTRUCTIONS = """You are able to send an emial."""

email_agent = Agent(
    name="Emailer agent",
    instructions=INSTRUCTIONS,
    tools=[send_html_email],
    model="gpt-4o-mini"
)



In [29]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [30]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def dispatch_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [48]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    print(f"{search_results}")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def prepare_email(report: ReportData):
    """ USing the email_writer_agent to make a data strcut that has subject and body"""
    result = await Runner.run(email_writer_agent, report )
    return result

async def send_email(input:EmailContent):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, input)
    print("Email sent")
    return

### Showtime!

In [52]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)  # inrerally calls the planner agentm which returns list[WebSearchItem]
    search_results = await dispatch_searches(search_plan)  # gets the list[WebSearchItem] and forces calling 'search_agent' on each (the one that uses WebSearchTool). results is a list of 3 large 300 word strings.
    report = await write_report(query, search_results) # internaly Calls await Runner.run(writer_agent, input), the input is basically an array of 3 large strings.
    email_content = await prepare_email(report.model_dump_json()) # calls the email writer agent to prepare the emai lsubject and body
    input_str = email_content.final_output.model_dump_json()
    await Runner.run(email_agent, input_str) # calls email_agent, which uses the send_html_tool
    print("Hooray!")


Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
["In 2025, several AI agent frameworks have been developed to enhance the capabilities and deployment of autonomous AI systems:\n\n- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models for any AI agent. It decouples agent execution from training, allowing integration with existing agents with minimal code modifications. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))\n\n- **GoalfyMax**: A protocol-driven multi-agent system introducing a standardized Agent-to-Agent communication layer and an Experience Pack architecture for structured knowledge retention and continual learning. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))\n\n- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI a

In [53]:
with trace("Test code trce"):
    email_content = await prepare_email(report.model_dump_json())
    print(email_content)
    input_str = email_content.final_output.model_dump_json()
    result = await Runner.run(email_agent, input_str)

RunResult:
- Last agent: Agent(name="Email Writer", ...)
- Final output (EmailContent):
    {
      "subject": "Insights and Analysis: AI Agent Frameworks in 2025",
      "message_body": "<html>\n<body>\n<h1>Latest AI Agent Frameworks in 2025</h1>\n\n<h2>Table of Contents</h2>\n\n<ol>\n<li><a href=\"#introduction\">Introduction</a></li>\n<li><a href=\"#evolution-of-ai-agents\">The Evolution of AI Agents</a></li>\n<li><a href=\"#key-ai-agent-frameworks\">Key AI Agent Frameworks</a>\n   <ul>\n   <li><a href=\"#agent-lightning\">Agent Lightning</a></li>\n   <li><a href=\"#goalfymax\">GoalfyMax</a></li>\n   <li><a href=\"#cognitive-kernel-pro\">Cognitive Kernel-Pro</a></li>\n   <li><a href=\"#agentscope-1-0\">AgentScope 1.0</a></li>\n   <li><a href=\"#openai-agents-sdk\">OpenAI Agents SDK</a></li>\n   <li><a href=\"#google-agent-development-kit-adk\">Google Agent Development Kit (ADK)</a></li>\n   <li><a href=\"#salesforce-agentforce-360\">Salesforce Agentforce 360</a></li>\n   <li><a href

### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>